# Image processing
# Lab III - Neighbourhood operations. Nonlinear Filtering

## 1. Nonlinear filters. Rank-order filters
The previous lab tackled the general aspects of neighbourhood operations and linear filtering, as an example. That particular kind of filter is suited in some denoising tasks. <b>In case an image is affected by Gaussian (normal) noise, it can be improved by applying a filter that averages the values in each neighbourhood.</b> Nonlinear filters are a class of filters where the output of the filter <b>is not a linear combination of other pixels.</b>

For different types of noise, linear filtering is not adequate. A different kind of noise is the <i>impulsive</i> noise (also called "salt and pepper" noise). This kind of noise affects only some pixels in the image, but completely corrupts the values of the pixels, by making them black (0) or white(1 or 255, depending if the image is scaled or not).
<img src="media/sp_lena.png"><center>Example of an image affected by impulsive noise</center>

In this case, the use of an average filter would only spread the noise to other pixels. For problems like these, a class of nonlinear filters called rank-order filters are best suited to solve the issue. This class of filters does not combine the values in the neighbourhood, instead selecting just one of the values in the neighbourhood using a certain criterion.

## 2. Algorithm and different types of rank-order filters
As was previously stated, rank-order filters select just one of the values of the neighbourhood, based on a decision process.  The main idea is to sort the values in the neighbourhood and pick the position needed to get the best suited pixel for our task. From the point of view of what value is selected, some examples of rank-order filters are:
 - min-filter, which selects the lowest value in the neighbourhood;
 - max-filter, which selects the highest value in the neighbourhood;
 - median filter, which selects the middle value.
 
The algorithm for the median filter is the following:
 - select pixel and neighbourhood
 - sort the pixels in the neighbourhood in an ascending order;
 - the value of the pixel in the output image is the one in the middle of the sorted vector of values (the fifth one, if the neighbourhood is 3x3);
 - repeat for every pixel (neigbhourhood) in the image.
 
<img src="media/median_filter.png"><center>Example of using the median filer on a neighbourhood</center>

### Exercise 1. Read an image (and convert it to grayscale). Add salt and pepper noise to the image. Apply the median filter to the image. Show the original noisy image, and the filtered one.

In order to add salt and pepper noise to a image we need an extra module from scikit-image, so change the import accordingly:
```python
from skimage import io,util,color
```
To add the noise, add this <b>just after you convert the image to grayscale</b>:
```python
img = util.random_noise(img,'s&p')
img = img * 255 # We still want to scale the values to 0...255
```

To sort in an ascending order the values of a matrix ```N```  into a vector, use numpy:
```python
N = np.sort(N,None)
```

In [ ]:
from skimage import io, color, util
import numpy as np
import matplotlib.pyplot as plt

plt.close('all')

img = io.imread('lena.png')

img = color.rgb2gray(img)
print(img.dtype)

img = np.uint8(img*256)

plt.figure(), plt.imshow(img, cmap='gray', vmin=0, vmax=256), plt.colorbar() #original image in B&W

img_n = util.random_noise(img, 's&p', amount = 0.05)

print(img_n.dtype)
print(img_n.shape)

img_n = np.round(img_n*256)

print(img_n.dtype)

plt.figure(), plt.imshow(img_n, cmap='gray'), plt.colorbar()

h,w, = img.shape

img_f = img.copy()

for i in range(1, h-1):
    for j in range(1, w-1):
        V = img_n[i-1:i+2, j-1:j+2]
        V = np.sort(V, axis=None)
        img_f[i,j]=V[4]

plt.figure(), plt.imshow(img_f, cmap='gray', vmin=0, vmax=256, label='Filter N'), plt.colorbar()
        
mask=np.ones([3,3])/9

img_f1 = img.copy()

for i in range(1, h-1):
    for j in range(1, w-1):
        V = img_n[i-1:i+2, j-1:j+2]
        V = V*mask
        img_f1[i,j]=np.sum(V)
        
plt.figure(), plt.imshow(img_f1, cmap='gray', vmin=0, vmax=256, label= 'Filter L'), 
plt.colorbar()

def MSE(original, processed):
    mse = np.sum(processed[1:h-1, 1:w-1] - original[1:h-1, 1:w-1])**2
    mse = mse/10**14
    return mse

mse_arithmetic = MSE(img, img_f)
mse_weighted = MSE(img, img_f1)
print(f"MSE for nonlinear filter: {mse_arithmetic}")
print(f"MSE for linear filter: {mse_weighted}")

### Exercise 2. Try to select different positions from the sorted vector (max, min, or other values between them) and see the how that impacts the noisy image. Test them on the original image (no noise) as well.

## RECAP (IMPORTANT)!!!
We have studied two types of noise:
 - Gaussian, which is aditive (the values of a gaussian distribution are added to the image). We studied it in the second lab, and to filter it, we need an average filter (linear filtering).
 <img src="media/gauss_lena.png">
 - Impulsive (salt and pepper), which corrupts the values of some pixels to extreme values. We studied it today, and to filter it, we need a median filter (nonlinear filtering).
 <img src="media/sp_lena.png">